# Proyecto Parte III - Feature Selection + Model

Autor: `Perez`  

Este notebook cumple con la consigna del Proyecto Final:

1. Elegir un método de *feature selection* para reducir la dimensionalidad.
2. Elegir un algoritmo de clasificación para entrenar con los datos elegidos.
3. Cálculo de métricas básicas para validar el modelo.
4. Generar conclusiones a partir de los resultados.

**Instrucciones de uso:** Coloca tu dataset en el mismo directorio con el nombre `data.csv` y una columna llamada `target`. Si no hay `data.csv`, el notebook generará un dataset sintético de ejemplo y lo guardará como `data.csv`.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectFromModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

print('librerías cargadas')

In [ ]:
# Cargar dataset
fn = 'data.csv'
if os.path.exists(fn):
    df = pd.read_csv(fn)
    print(f'Archivo {fn} encontrado — shape:', df.shape)
else:
    print(f'Archivo {fn} no encontrado. Generando dataset sintético...')
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples=1000, n_features=25, n_informative=8, n_redundant=5,
                               n_repeated=0, n_classes=2, random_state=42)
    cols = [f'feat_{i}' for i in range(X.shape[1])]
    df = pd.DataFrame(X, columns=cols)
    df['target'] = y
    df.to_csv(fn, index=False)
    print(f'Dataset sintético guardado como {fn} — shape:', df.shape)

# Mostrar primeras filas
display(df.head())
print('\nColumnas:', df.columns.tolist())

## Exploración básica de los datos

In [ ]:
# Resumen rápido
print('Nulos por columna:')
print(df.isna().sum())
print('\nDistribución del target:')
print(df['target'].value_counts(normalize=True))

# Describir numéricos
print('\nDescripción estadística:')
print(df.describe().T)

## Preparar matrices X, y y partición train/test

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

## Método de Feature Selection 1 — SelectKBest (mutual information)
Elegimos las 10 mejores características según `mutual_info_classif` (útil para relaciones no lineales).

In [ ]:
k = 10
skb = SelectKBest(score_func=mutual_info_classif, k=k)
skb.fit(X_train, y_train)
support_skb = skb.get_support()
selected_kbest = X_train.columns[support_skb].tolist()
print(f'Se seleccionaron {len(selected_kbest)} features con SelectKBest:')
print(selected_kbest)

# guardar transformaciones
X_train_kbest = skb.transform(X_train)
X_test_kbest = skb.transform(X_test)

## Método de Feature Selection 2 — SelectFromModel (RandomForest feature importances)
Usamos un RandomForest rápido para estimar importancias y seleccionar features.

In [ ]:
rf_for_fs = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_for_fs.fit(X_train, y_train)
selector = SelectFromModel(rf_for_fs, prefit=True, threshold='median')
support_sfm = selector.get_support()
selected_sfm = X_train.columns[support_sfm].tolist()
print('Features seleccionadas por SelectFromModel (threshold=median):')
print(selected_sfm)

X_train_sfm = selector.transform(X_train)
X_test_sfm = selector.transform(X_test)

In [ ]:
print('KBest selected:', len(selected_kbest))
print('SelectFromModel selected:', len(selected_sfm))

## Entrenar modelo — RandomForestClassifier
Probamos entrenar el mismo algoritmo usando las dos selecciones de features y también con todas las features para comparar.

In [ ]:
def train_and_eval(clf, X_tr, X_te, y_tr, y_te, cv=5):
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    scores = {
        'accuracy': accuracy_score(y_te, y_pred),
        'precision': precision_score(y_te, y_pred),
        'recall': recall_score(y_te, y_pred),
        'f1': f1_score(y_te, y_pred)
    }
    print('Test scores:', scores)
    print('\nClassification report:\n', classification_report(y_te, y_pred))
    cm = confusion_matrix(y_te, y_pred)
    return scores, cm

rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)

print('--- Usando todas las features ---')
scores_all, cm_all = train_and_eval(rf, X_train, X_test, y_train, y_test)

print('\n--- Usando SelectKBest (k=', len(selected_kbest),') ---')
scores_kbest, cm_kbest = train_and_eval(rf, X_train_kbest, X_test_kbest, y_train, y_test)

print('\n--- Usando SelectFromModel (', len(selected_sfm), 'features) ---')
scores_sfm, cm_sfm = train_and_eval(rf, X_train_sfm, X_test_sfm, y_train, y_test)

## Validación cruzada (5-fold) — comparar rendimiento estable
Usamos `StratifiedKFold` para medir estabilidad con las tres configuraciones.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def cv_score(clf, X, y):
    s = cross_val_score(clf, X, y, cv=cv, scoring='f1')
    return s.mean(), s.std()

mean_all, std_all = cv_score(rf, X, y)
mean_kbest, std_kbest = cv_score(rf, skb.transform(X), y)
mean_sfm, std_sfm = cv_score(rf, selector.transform(X), y)

print('F1 mean (all features):', mean_all, '±', std_all)
print('F1 mean (SelectKBest):', mean_kbest, '±', std_kbest)
print('F1 mean (SelectFromModel):', mean_sfm, '±', std_sfm)

## Matrices de confusión (test set)

In [ ]:
def plot_cm(cm, title='Confusion matrix'):
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha='center', va='center')
    plt.show()

plot_cm(cm_all, 'Confusion matrix - All features')
plot_cm(cm_kbest, 'Confusion matrix - SelectKBest')
plot_cm(cm_sfm, 'Confusion matrix - SelectFromModel')

## Importancias de features (RandomForest)
Mostramos las 15 features más importantes según el RandomForest usado para entrenamiento de features.

In [ ]:
# usar el modelo rf_for_fs entrenado antes
importances = rf_for_fs.feature_importances_
feat_names = X_train.columns
imp_df = pd.DataFrame({'feature': feat_names, 'importance': importances}).sort_values('importance', ascending=False)
imp_df_top = imp_df.head(15)
print(imp_df_top)

plt.figure()
plt.bar(range(len(imp_df_top)), imp_df_top['importance'])
plt.xticks(range(len(imp_df_top)), imp_df_top['feature'], rotation=90)
plt.title('Top 15 feature importances (RandomForest)')
plt.tight_layout()
plt.show()

## Conclusiones

- Se presentaron dos métodos de selección de características: `SelectKBest` (mutual information) y `SelectFromModel` (basado en importancias de RandomForest).
- Comparando las tres configuraciones (todas las features, SelectKBest, SelectFromModel) observamos la diferencia en rendimiento en el set de test y mediante validación cruzada.  
- Dependiendo del dataset real, una reducción de dimensionalidad puede mantener el rendimiento mientras reduce costo computacional y mejorar interpretabilidad.  

**Recomendaciones para entrega:**
1. Subir este notebook `/mnt/data/ProyectoParteIII_Perez.ipynb` a tu repositorio GitHub junto con tu `data.csv` real.
2. Asegúrate de que el notebook presente: código compactado por secciones, resultados (tablas/plots) y conclusiones.
3. Si tu problema es regresión (en lugar de clasificación), cambia el clasificador por `RandomForestRegressor` y las métricas por MAE/MSE/R2.

¡Listo!

El archivo ha sido guardado como `/mnt/data/ProyectoParteIII_Perez.ipynb`. Descárgalo desde el enlace proporcionado por el entorno.